<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST2312_H11/blob/main/CST2312_FOPP_Ch10_CSV_Files_wSolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Working with CSV Files**

*by Professor Patrick — 2026*

*a copy of this Colab Jupyter notebook is [available online](https://bit.ly/cst2312FOPPch10)*

**Textbook Reference:** Runestone FOPP, [Chapter 10 — Files](https://runestone.academy/ns/books/published/fopp/Files/intro-WorkingwithDataFiles.html)
**Course:** CST2312 — Information and Data Management

### Learning Objectives

By the end of this session you will be able to:

1. Describe the structure and conventions of the CSV (Comma-Separated Values) file format.
2. Read CSV data from a file by splitting each line on the delimiter.
3. Use Python's built-in `csv` module to read CSV files with `csv.reader`.
4. Process CSV rows using the accumulator pattern to extract and transform data.
5. Write data to a CSV file using both manual string formatting and `csv.writer`.
6. Handle common CSV issues including headers, type conversion, and fields containing commas.

### FOPP Chapter 10 Section Map

| Notebook Segment | FOPP Section(s) |
|:---|:---|
| What Is a CSV File? | 10.9 CSV Format |
| Reading CSV Data (Manual Approach) | 10.10 Reading in data from a CSV File |
| Reading CSV Data (csv Module) | 10.10 |
| Writing CSV Data | 10.11 Writing data to a CSV File |
| Prerequisites Review (from Session 1) | 9.8 Append versus Concatenate, 9.12 The Accumulator Pattern with Lists |
| Self-Study (reading assignment) | 10.1–10.8 (basic file operations), 10.12 Tips on Handling Files |

---

## Setup: Create Sample Data Files

Since we are working in Google Colab, we do not have local files on disk. The cell below creates sample CSV files that we will use throughout this notebook. **Run this cell first** before proceeding.

In [ ]:
# ============================================================
# RUN THIS CELL FIRST — creates sample CSV files for the lesson
# ============================================================

# --- Sample 1: students.csv (simple, no commas in fields) ---
students_csv = """Name,Major,GPA
Alice Johnson,Computer Science,3.8
Bob Smith,Information Systems,3.5
Charlie Davis,Data Science,3.9
Diana Lee,Computer Science,3.2
Ethan Brown,Information Systems,3.7
"""

with open("students.csv", "w") as f:
    f.write(students_csv.strip())

print("Created: students.csv")

# --- Sample 2: sales.csv (numeric data for processing) ---
sales_csv = """Product,Quarter,Units,Price
Widget A,Q1,150,12.99
Widget A,Q2,200,12.99
Widget B,Q1,80,24.50
Widget B,Q2,120,24.50
Widget C,Q1,300,5.75
Widget C,Q2,350,5.75
"""

with open("sales.csv", "w") as f:
    f.write(sales_csv.strip())

print("Created: sales.csv")

# --- Sample 3: addresses.csv (fields containing commas) ---
addresses_csv = """Name,Address,City,State
Alice Johnson,"123 Main St, Apt 4",New York,NY
Bob Smith,456 Oak Ave,Los Angeles,CA
Charlie Davis,"789 Pine Rd, Suite 200",Chicago,IL
"""

with open("addresses.csv", "w") as f:
    f.write(addresses_csv.strip())

print("Created: addresses.csv")
print()
print("All sample files are ready.")

---

## What Is a CSV File?
*(FOPP 10.9 — 8 min)*

**CSV** stands for **Comma-Separated Values**. It is one of the simplest and most widely used formats for storing tabular data — data organized in rows and columns, like a spreadsheet.

A CSV file is a plain text file where:
- Each **line** represents one **row** (record).
- Values within a row are separated by a **delimiter**, most commonly a comma.
- The **first line** often contains column headers (field names), though this is a convention, not a requirement.

**Example — `students.csv`:**
```
Name,Major,GPA
Alice Johnson,Computer Science,3.8
Bob Smith,Information Systems,3.5
Charlie Davis,Data Science,3.9
```

### Why CSV?

CSV files are ubiquitous because they are:
- **Human-readable** — you can open them in any text editor.
- **Tool-agnostic** — virtually every data tool (Excel, Google Sheets, databases, Python, R) can read and write CSV.
- **Lightweight** — no formatting overhead, just raw data.

They are also limited: CSV has no standard way to represent data types (everything is text), and fields containing commas or newlines require special handling (quoting).

In [ ]:
# Let us look at the raw contents of students.csv
with open("students.csv", "r") as f:
    raw_text = f.read()

print(raw_text)
print()
print("Each line is one record. Commas separate the fields.")

In [ ]:
# Read line by line to see the structure
with open("students.csv", "r") as f:
    for line_number, line in enumerate(f):
        print(f"Line {line_number}: {repr(line.strip())}")

---

## Reading CSV Data: The Manual Approach
*(FOPP 10.10 — 12 min)*

The most basic way to read a CSV file is to combine file reading techniques from earlier in Chapter 10 with the `split()` method from Chapter 6 (Section 6.9).

**The pattern:**
1. Open the file.
2. Read line by line.
3. Strip whitespace/newline characters from each line.
4. Split each line on the comma delimiter.
5. Process the resulting list of fields.

This approach uses the **accumulator pattern** from the previous session to build up a list of processed rows.

In [ ]:
# Manual CSV reading: split each line on commas
with open("students.csv", "r") as f:
    header_line = f.readline().strip()  # Read the header row separately
    headers = header_line.split(",")
    print("Headers:", headers)
    print()

    students = []                        # Accumulator
    for line in f:                       # Iterate over remaining lines
        fields = line.strip().split(",")
        students.append(fields)          # Accumulate

# Display the result
for student in students:
    print(student)

### Processing Numeric Fields

All values read from a CSV file arrive as **strings**. If you need to perform arithmetic on numeric fields, you must convert them explicitly using `int()` or `float()`.

*(Recall FOPP 2.6: Type conversion functions)*

In [ ]:
# Read sales data and compute revenue per row
with open("sales.csv", "r") as f:
    headers = f.readline().strip().split(",")
    print(f"{'Product':<12} {'Quarter':<10} {'Units':>6} {'Price':>8} {'Revenue':>10}")
    print("-" * 50)

    total_revenue = 0                     # Numeric accumulator
    for line in f:
        fields = line.strip().split(",")
        product = fields[0]
        quarter = fields[1]
        units   = int(fields[2])          # Convert to int
        price   = float(fields[3])        # Convert to float
        revenue = units * price

        total_revenue += revenue
        print(f"{product:<12} {quarter:<10} {units:>6} {price:>8.2f} {revenue:>10.2f}")

print("-" * 50)
print(f"{'Total Revenue':>38} {total_revenue:>10.2f}")

### Limitation of the Manual Approach

Splitting on commas works for simple CSV files, but it **fails** when a field itself contains a comma. For example, an address field like `"123 Main St, Apt 4"` would be split incorrectly into two separate fields.

Let us verify this problem:

In [ ]:
# Demonstrate the problem with commas inside fields
with open("addresses.csv", "r") as f:
    headers = f.readline().strip().split(",")
    print("Headers:", headers)
    print(f"Expected columns: {len(headers)}")
    print()

    for line in f:
        fields = line.strip().split(",")
        print(f"Fields ({len(fields)}): {fields}")

print()
print("PROBLEM: The address field containing a comma was split incorrectly.")
print("Row 1 has 5 fields instead of 4.")

---

## Reading CSV Data: The `csv` Module
*(FOPP 10.10 — 10 min)*

Python's built-in `csv` module solves the quoting problem and provides a reliable way to read CSV files correctly. It handles:
- Fields enclosed in quotes (which may contain commas or newlines).
- Different delimiters (tabs, semicolons, pipes).
- Consistent row parsing regardless of field content.

The primary tool is `csv.reader`, which wraps a file object and yields each row as a list of strings.

In [ ]:
import csv

# csv.reader handles quoted fields correctly
with open("addresses.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)     # Read the header row
    print("Headers:", headers)
    print()

    for row in reader:
        print(f"Fields ({len(row)}): {row}")

print()
print("The csv module correctly parsed the quoted address fields.")

### Using `csv.reader` with the Accumulator Pattern

The combination of `csv.reader` and the accumulator pattern is the standard approach for loading CSV data into a list of lists for further processing.

In [ ]:
import csv

# Load all student records into a list of lists
with open("students.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)        # Separate the header row

    all_students = []             # Accumulator
    for row in reader:
        all_students.append(row)

print("Headers:", headers)
print(f"Loaded {len(all_students)} student records.")
print()

for student in all_students:
    print(f"  Name: {student[0]:<20} Major: {student[1]:<25} GPA: {student[2]}")

### Filtering with `csv.reader` and the Accumulator

Combine `csv.reader` with a conditional inside the loop to select specific rows.

In [ ]:
import csv

# Find students with GPA above 3.6
with open("students.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)

    honor_roll = []
    for row in reader:
        gpa = float(row[2])               # Convert GPA to float
        if gpa > 3.6:
            honor_roll.append(row)

print("Honor Roll (GPA > 3.6):")
for student in honor_roll:
    print(f"  {student[0]} — {student[1]} — GPA: {student[2]}")

---

## Writing CSV Data
*(FOPP 10.11 — 10 min)*

Just as there are manual and module-based approaches to reading CSV, there are two approaches to writing.

### Manual Approach: String Formatting

You can construct each line as a comma-separated string and write it to a file.

In [ ]:
# Manual CSV writing using string join
grades = [
    ["Alice Johnson", "A", "95"],
    ["Bob Smith", "B+", "87"],
    ["Charlie Davis", "A+", "98"],
    ["Diana Lee", "B", "83"],
]

with open("grades_output.csv", "w") as f:
    f.write("Name,Grade,Score\n")          # Write header
    for row in grades:
        line = ",".join(row)               # Join fields with commas
        f.write(line + "\n")

# Verify the output
print("Contents of grades_output.csv:")
print()
with open("grades_output.csv", "r") as f:
    print(f.read())

### Using `csv.writer`

The `csv.writer` object handles quoting automatically. If a field contains a comma, it will be enclosed in quotes. This is the recommended approach for producing correct CSV output.

In [ ]:
import csv

# Using csv.writer for reliable output
products = [
    ["Widget A", "Small, lightweight", 12.99],
    ["Widget B", "Medium, durable", 24.50],
    ["Widget C", "Budget option", 5.75],
]

with open("products_output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Product", "Description", "Price"])   # Write header
    for row in products:
        writer.writerow(row)                               # Write each data row

# Verify — notice how the csv module quoted fields containing commas
print("Contents of products_output.csv:")
print()
with open("products_output.csv", "r") as f:
    print(f.read())

**Note:** The `newline=""` parameter in the `open()` call is important on Windows systems to prevent extra blank lines in the output. It is harmless on other platforms and is considered best practice to include it whenever writing CSV files.

### Read, Process, Write — A Complete Pipeline

A common real-world task is to read data from one CSV, process or filter it, and write the results to a new CSV. This combines everything from today's session and the previous one.

In [ ]:
import csv

# Pipeline: Read sales data, compute revenue, write results to a new file
with open("sales.csv", "r") as infile, \
     open("sales_with_revenue.csv", "w", newline="") as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Read and write headers (adding a new column)
    headers = next(reader)
    headers.append("Revenue")
    writer.writerow(headers)

    # Process each row
    for row in reader:
        units   = int(row[2])
        price   = float(row[3])
        revenue = units * price
        row.append(f"{revenue:.2f}")
        writer.writerow(row)

# Verify the output
print("Contents of sales_with_revenue.csv:")
print()
with open("sales_with_revenue.csv", "r") as f:
    print(f.read())

---

## Exercises: Working with CSV Files

**Objective:** Practice reading, processing, and writing CSV data using both manual techniques and the `csv` module.

### Task 1: Read and Display CSV Data

Read `students.csv` using `csv.reader`. Print each student's name and GPA in the following format:

```
Alice Johnson has a GPA of 3.8
Bob Smith has a GPA of 3.5
...
```

**Starter code is provided below.** Fill in the body of the loop.

**Expected output:**
```
Alice Johnson has a GPA of 3.8
Bob Smith has a GPA of 3.5
Charlie Davis has a GPA of 3.9
Diana Lee has a GPA of 3.2
Ethan Brown has a GPA of 3.7
```

In [ ]:
# Task 1: Read and display student data
import csv

with open("students.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)        # Skip the header row

    for row in reader:
        # Your code here: print the formatted string using row[0] and row[2]
        pass

### Task 2: Compute Average GPA

Read `students.csv` using `csv.reader`. Use **two accumulators** — one for the sum of all GPAs and one for the count of students — to compute and print the average GPA.

**Hint:** Convert each GPA from a string to a `float` before adding it to the sum.

**Expected output:**
```
Total students: 5
Average GPA: 3.62
```

In [ ]:
# Task 2: Compute average GPA
import csv

total_gpa = 0       # Accumulator for sum
count = 0           # Accumulator for count

with open("students.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)

    for row in reader:
        # Your code here: convert GPA to float, add to total_gpa, increment count
        pass

# Your code here: compute and print the average
# print(f"Total students: {count}")
# print(f"Average GPA: {average:.2f}")

### Task 3: Filter and Write to a New File

Read `sales.csv` using `csv.reader`. Filter for rows where the number of units sold is **greater than 100**. Write only those rows (with the original headers) to a new file called `high_volume_sales.csv`.

**Hint:** Use the accumulator pattern to collect qualifying rows, then write them with `csv.writer`.

**Expected output file contents:**
```
Product,Quarter,Units,Price
Widget A,Q1,150,12.99
Widget A,Q2,200,12.99
Widget B,Q2,120,24.50
Widget C,Q1,300,5.75
Widget C,Q2,350,5.75
```

In [ ]:
# Task 3: Filter high-volume sales and write to new CSV
import csv

# Your code here:
# 1. Read sales.csv with csv.reader
# 2. Save the header row
# 3. Use accumulator pattern to collect rows where int(row[2]) > 100
# 4. Write headers and filtered rows to high_volume_sales.csv with csv.writer
# 5. Read back and print the new file to verify

### Task 4 (Challenge): Build a Summary Report

Read `sales.csv` and produce a summary showing **total units** and **total revenue** for each product. Write the summary to `product_summary.csv`.

**Hint:** Use a dictionary as your accumulator, where each key is a product name and each value is a list `[total_units, total_revenue]`.

**Expected output file contents:**
```
Product,Total Units,Total Revenue
Widget A,350,4546.50
Widget B,200,4900.00
Widget C,650,3737.50
```

In [ ]:
# Task 4 (Challenge): Build a product summary report
import csv

# Your code here:
# 1. Read sales.csv
# 2. Accumulate totals per product using a dictionary
# 3. Write the summary to product_summary.csv
# 4. Print the file to verify

### Task 5 (Challenge): Handle a CSV with Commas in Fields

Read `addresses.csv` using `csv.reader` (not manual splitting). For each row, print the person's name and their complete address (combining the `Address`, `City`, and `State` fields).

**Expected output:**
```
Alice Johnson: 123 Main St, Apt 4, New York, NY
Bob Smith: 456 Oak Ave, Los Angeles, CA
Charlie Davis: 789 Pine Rd, Suite 200, Chicago, IL
```

**Why does this exercise matter?** It demonstrates precisely why `csv.reader` is preferred over manual `split(",")` — the quoted address fields are parsed correctly.

In [ ]:
# Task 5 (Challenge): Reading CSV with commas inside quoted fields
import csv

# Your code here: read addresses.csv and print formatted output

---

**Congratulations!**

You have read and written CSV files using both manual string operations and Python's `csv` module. You have applied the accumulator pattern from the previous session to filter, transform, and aggregate tabular data. These skills form the foundation for working with data in more advanced libraries such as Pandas.

---

## Glossary
*(FOPP 10.13)*

- **CSV (Comma-Separated Values)** — A plain text file format for storing tabular data, where each line is a row and fields within a row are separated by commas (or another delimiter).
- **csv module** — Python's built-in library for reading and writing CSV files correctly, including proper handling of quoted fields.
- **csv.reader** — A function that wraps a file object and yields each row as a list of strings, correctly parsing quoted fields and delimiters.
- **csv.writer** — A function that wraps a file object and provides `writerow()` and `writerows()` methods for writing lists as properly formatted CSV rows.
- **delimiter** — The character used to separate fields in a CSV file. The comma is the default, but tabs (`\t`), semicolons (`;`), and pipes (`|`) are also common.
- **header row** — The first line of a CSV file, which typically contains column names (field names). Read separately using `next(reader)` or `readline()`.
- **newline parameter** — The `newline=""` argument to `open()` when writing CSV files, which prevents the insertion of extra blank lines on Windows.
- **quoting** — The practice of enclosing a CSV field in double quotes when it contains the delimiter character, a newline, or a quote character itself.

---

## Reading Assignment

- **Runestone FOPP Chapter 10**, Sections 10.1 through 10.14 (review the embedded interactive exercises as you read)
- Complete the **Chapter 10 Exercises** (Section 10.14)

The following sections are designated for self-study and will appear on assessments:

- **10.1** Introduction: Working with Data Files
- **10.2** Reading a File
- **10.3** Alternative File Reading Methods
- **10.4** Iterating over lines in a file
- **10.5** Finding a File in your Filesystem
- **10.6** Using `with` for Files
- **10.7** Recipe for Reading and Processing a File
- **10.8** Writing Text Files
- **10.12** Tips on Handling Files

---

## *Solutions*

### *Task 1 — Solution: Read and Display CSV Data*

In [ ]:
# Task 1 Solution
import csv

with open("students.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)

    for row in reader:
        print(f"{row[0]} has a GPA of {row[2]}")

### *Task 2 — Solution: Compute Average GPA*

In [ ]:
# Task 2 Solution
import csv

total_gpa = 0
count = 0

with open("students.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)

    for row in reader:
        total_gpa += float(row[2])
        count += 1

average = total_gpa / count
print(f"Total students: {count}")
print(f"Average GPA: {average:.2f}")

### *Task 3 — Solution: Filter and Write to a New File*

In [ ]:
# Task 3 Solution
import csv

# Read and filter
with open("sales.csv", "r") as infile:
    reader = csv.reader(infile)
    headers = next(reader)

    high_volume = []
    for row in reader:
        if int(row[2]) > 100:
            high_volume.append(row)

# Write filtered data
with open("high_volume_sales.csv", "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(headers)
    for row in high_volume:
        writer.writerow(row)

# Verify
print("Contents of high_volume_sales.csv:")
print()
with open("high_volume_sales.csv", "r") as f:
    print(f.read())

### *Task 4 — Solution: Build a Summary Report*

In [ ]:
# Task 4 Solution
import csv

# Read and accumulate per-product totals using a dictionary
product_totals = {}

with open("sales.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # Skip header

    for row in reader:
        product = row[0]
        units   = int(row[2])
        price   = float(row[3])
        revenue = units * price

        if product not in product_totals:
            product_totals[product] = [0, 0.0]

        product_totals[product][0] += units
        product_totals[product][1] += revenue

# Write summary
with open("product_summary.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Product", "Total Units", "Total Revenue"])
    for product, totals in product_totals.items():
        writer.writerow([product, totals[0], f"{totals[1]:.2f}"])

# Verify
print("Contents of product_summary.csv:")
print()
with open("product_summary.csv", "r") as f:
    print(f.read())

### *Task 5 — Solution: Handle CSV with Commas in Fields*

In [ ]:
# Task 5 Solution
import csv

with open("addresses.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # Skip header

    for row in reader:
        name    = row[0]
        address = row[1]
        city    = row[2]
        state   = row[3]
        print(f"{name}: {address}, {city}, {state}")

---